In [71]:
import recordlinkage
import pandas as pd
import pickle

In [72]:
config_path = '/home/edkl/project/20222/THDL/BTL/THDL/config_mapping'

In [75]:
with open(config_path + '/muaban_okxe.pkl', 'rb') as f:
    muaban_okxe = pickle.load(f)
with open(config_path + '/muaban_webike.pkl', 'rb') as f:
    muaban_webike = pickle.load(f)
with open(config_path + '/muaban_xemayhoangkien.pkl', 'rb') as f:
    muaban_xemayhoangkien = pickle.load(f)

In [99]:
xemayhoangkien_inv = {value: key for (key, value) in muaban_xemayhoangkien.items()}
okxe_inv = {value: key for (key, value) in muaban_okxe.items()}
webike_inv = {value: key for (key, value) in muaban_webike.items()}

In [100]:
xemayhoangkien_inv

{'price': 'price',
 'name': 'name',
 'url': 'url',
 'Năm đăng ký': 'Năm đăng ký',
 'Màu sắc': 'Màu xe',
 'Dung tích': 'Dung tích xe máy',
 'Thương hiệu': 'Hãng xe',
 'Loại': 'Loại xe máy'}

In [3]:
data_folder = "../crawl/crawl/spiders/all_data"
okxe_data = pd.read_csv(data_folder + "/okxe.csv").drop(columns=['Unnamed: 0'], axis=1)
muaban_data = pd.read_csv(data_folder + "/muaban.csv").drop(columns=['Unnamed: 0'], axis=1)
webike_data = pd.read_csv(data_folder + "/webike.csv").drop(columns=['Unnamed: 0'], axis=1)
xemayhoangkien_data = pd.read_csv(data_folder + "/xemayhoangkien.csv").drop(columns=['Unnamed: 0'], axis=1)

In [ ]:
xemayhoangkien_data.head()

In [ ]:
muaban_data.head()

In [ ]:
webike_data.head()

In [ ]:
okxe_data.head()

In [104]:
all = pd.DataFrame(columns=['name', 'price', 'address', 'url', 'brand', 'registerYear', 'status', 'type', 'origin', 'capacity', 'color', 'vehicleType', 'km', 'imageUrl'])
all

,name,price,address,url,brand,registerYear,status,type,origin,capacity,color,vehicleType,km,imageUrl


In [94]:
# Build the indexer
indexer = recordlinkage.Index()
# Can use full or block
# indexer.full()
# indexer.block(left_on='State', right_on='Provider State')

# Use sortedneighbor as a good option if data is not clean
indexer.sortedneighbourhood(left_on='name', right_on='name')
candidates = indexer.index(muaban_data, webike_data)
compare = recordlinkage.Compare()
# compare.exact('City', 'Provider City', label='City')
compare.string('name','name', method='levenshtein',threshold=0.9,label='Name')
compare.string('Dòng xe', 'Dòng xe', method='jarowinkler', threshold=0.5, label='Dòng xe')
compare.string('address', 'address', method='jarowinkler', threshold=0.5, label='address')
compare.string('Loại xe máy', 'Loại xe', method='jarowinkler', threshold=0.5, label='Loại xe')
compare.exact('Số km đã đi', 'Đã đi', label='Đã đi')
features = compare.compute(candidates, muaban_data,
                           webike_data)
# Get the potential matches
potential_matches = features[features.sum(axis=1) > 3].reset_index()
potential_matches

,level_0,level_1,Name,Dòng xe,address,Loại xe,Đã đi
0,436,1545,1.0,1.0,1.0,1.0,0
1,187,1545,1.0,1.0,1.0,1.0,0


In [98]:
webike_data[~webike_data.index.isin(potential_matches['level_1'])]

,name,price,address,url,Dòng xe,Đã đi,Xuất xứ,Đời xe,Phân khối,Loại xe,Màu,Ngày đăng
1544,Bán xe Rebel 300 độ full,124000000.0,"Thanh Khê , Đà Nẵng",https://www.webike.vn/cho-xe-may/bike-detail/b...,HONDA khác,0 - 4 999 km,-,-,-,Khác,-,4 tháng trước
1546,Vision 2020 phiên bản đặc biệt khoá smartkey,24600000.0,-,https://www.webike.vn/cho-xe-may/bike-detail/v...,-,-,-,-,-,-,-,-
1547,Thanh Motor cần bán Kawasaki Z1000 2015,233000000.0,"Hoàng Mai , Hà Nội",https://www.webike.vn/cho-xe-may/bike-detail/t...,Kawasaki Z1000 ABS 2015,15 000 - 19 999 km,-,2015,1000 cc,Naked bike,-,4 tháng trước
1548,Bán Sh Mode Bản ĐB cuối 2017 Xe lướt Chính Chủ,48000000.0,"Thủ Đức , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/b...,HONDA SH Mode 125 2017,5 000 - 9 999 km,-,2017,125 cc,Xe tay ga,-,4 tháng trước
1549,không tiền bán kiếm tiền ăn Tết,3000000.0,-,https://www.webike.vn/cho-xe-may/bike-detail/k...,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Sprint S125 Iget Abs Màu Xanh Sport 2021,70000000.0,"Quận 5 , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/s...,PIAGGIO khác,5 000 - 9 999 km,-,-,-,Khác,-,4 tháng trước
2996,Xe Dream 2000 - Cần bán HONDA khác,11500000.0,"Hóc Môn , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/x...,HONDA khác,30 000 - 34 999 km,-,-,-,Khác,-,4 tháng trước
2997,raider đen nhám biển 71 chính chủ,25500000.0,"Bình Thạnh , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/r...,SUZUKI Raider R150 2018,0 - 4 999 km,-,2018,150 cc,Xe côn tay,-,4 tháng trước
2998,Virago 125 HQCN up full Cafe Racer,19000000.0,-,https://www.webike.vn/cho-xe-may/bike-detail/v...,-,-,-,-,-,-,-,-


In [36]:
dict_field = {
    'name': 'name', 
    'price': 'price', 
    'address': 'address', 
    'url': 'url', 
    'brand': 'Hãng xe', 
    'registerYear': 'Năm đăng ký', 
    'status': 'Tình trạng', 
    'type': 'Loại xe máy', 
    'origin': 'Xuất xứ', 
    'capacity': 'Dung tích xe máy', 
    'color': 'Màu xe', 
    'vehicleType': 'Dòng xe', 
    'km': 'Số km đã đi', 
    'imageUrl': 'image_url'
}

In [37]:
inverse_dict_field = {value: key for (key, value) in dict_field.items()}

In [106]:
all = all.append(muaban_data.rename(columns=inverse_dict_field))

/tmp/ipykernel_55707/2683210262.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(muaban_data.rename(columns=inverse_dict_field))


In [107]:
all

,name,price,address,url,brand,registerYear,status,type,origin,capacity,color,vehicleType,km,imageUrl
0,CẦN BÁN XE MÁY HONDA VISION 110 ĐỜI 2022,15000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,Đen,-,0,NaN
1,"Chuyên thu mua các loại xe máy cũ và mới, xe h...",0,"Quận Đống Đa, Hà Nội",https://muaban.net/xe-may/mua-xe-may/chuyen-th...,-,-,-,-,-,-,-,-,0,NaN
2,HEAD SƠN TÙNG 1 - TRỢ GIÁ MUA XE TƯNG BỪNG-VUI...,31990000,"Huyện Hoài Đức, Hà Nội",https://muaban.net/xe-may/ban-xe-may/head-son-...,Honda,-,Mới,Tay côn/Moto,-,-,-,-,0,NaN
3,Cần thanh lý lô xe honda AirBlade 150 nhập khẩu,19000000,"TP. Thủ Đức - Quận 9, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-thanh...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,Đen,Air Blade,0,NaN
4,CẦN BÁN XE MÁY HONDA SH150 ĐỜI 2022,32000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2023.0,Mới,Tay ga,Nước khác,101 - 175 cc,Ghi/Xám,SH,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,Suzuki Saphia máy êm ru chạy ngon như sapcy,4500000,-,https://muaban.net/xe-may/ban-xe-may-quan-5-ho...,-,-,-,-,-,-,-,-,0,NaN
1442,Mua Bảo Hiểm Yamaha chính hiệu kèm theo xe-giá...,180000,-,https://muaban.net/xe-may/phu-tung-xe-may/mua-...,-,-,-,-,-,-,-,-,0,NaN
1443,"Gia Công Xi Mạ Chất Lượng, Giá Hợp Lý, Bảo Hà...",5000,-,https://muaban.net/xe-may/phu-tung-xe-may/gia-...,-,-,-,-,-,-,-,-,0,NaN
1444,"AE cơ khí cần nhuộm đen, xi đen chiều dài 2m, ...",10000,-,https://muaban.net/xe-may/phu-tung-xe-may/ae-c...,-,-,-,-,-,-,-,-,0,NaN


In [112]:
all = all.append(webike_data[~webike_data.index.isin(potential_matches['level_1'])].rename(columns=webike_inv).rename(columns=inverse_dict_field))

/tmp/ipykernel_55707/3389037095.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(webike_data[~webike_data.index.isin(potential_matches['level_1'])].rename(columns=webike_inv).rename(columns=inverse_dict_field))


In [113]:
all

,name,price,address,url,brand,registerYear,status,type,origin,capacity,color,vehicleType,km,imageUrl,Đời xe,Phân khối
0,CẦN BÁN XE MÁY HONDA VISION 110 ĐỜI 2022,15000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,Đen,-,0,NaN,NaN,NaN
1,"Chuyên thu mua các loại xe máy cũ và mới, xe h...",0,"Quận Đống Đa, Hà Nội",https://muaban.net/xe-may/mua-xe-may/chuyen-th...,-,-,-,-,-,-,-,-,0,NaN,NaN,NaN
2,HEAD SƠN TÙNG 1 - TRỢ GIÁ MUA XE TƯNG BỪNG-VUI...,31990000,"Huyện Hoài Đức, Hà Nội",https://muaban.net/xe-may/ban-xe-may/head-son-...,Honda,-,Mới,Tay côn/Moto,-,-,-,-,0,NaN,NaN,NaN
3,Cần thanh lý lô xe honda AirBlade 150 nhập khẩu,19000000,"TP. Thủ Đức - Quận 9, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-thanh...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,Đen,Air Blade,0,NaN,NaN,NaN
4,CẦN BÁN XE MÁY HONDA SH150 ĐỜI 2022,32000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2023.0,Mới,Tay ga,Nước khác,101 - 175 cc,Ghi/Xám,SH,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Sprint S125 Iget Abs Màu Xanh Sport 2021,70000000.0,"Quận 5 , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/s...,NaN,4 tháng trước,NaN,Khác,-,NaN,-,PIAGGIO khác,5 000 - 9 999 km,NaN,-,-
2996,Xe Dream 2000 - Cần bán HONDA khác,11500000.0,"Hóc Môn , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/x...,NaN,4 tháng trước,NaN,Khác,-,NaN,-,HONDA khác,30 000 - 34 999 km,NaN,-,-
2997,raider đen nhám biển 71 chính chủ,25500000.0,"Bình Thạnh , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/r...,NaN,4 tháng trước,NaN,Xe côn tay,-,NaN,-,SUZUKI Raider R150 2018,0 - 4 999 km,NaN,2018,150 cc
2998,Virago 125 HQCN up full Cafe Racer,19000000.0,-,https://www.webike.vn/cho-xe-may/bike-detail/v...,NaN,-,NaN,-,-,NaN,-,-,-,NaN,-,-


In [114]:
candidates = indexer.index(muaban_data, okxe_data)

In [115]:
muaban_okxe

{'price': 'price',
 'name': 'name',
 'address': 'address',
 'url': 'url',
 'Tình trạng': 'used_status',
 'Hãng xe': 'brand'}

In [139]:
# Build the indexer
indexer = recordlinkage.Index()
# Can use full or block
# indexer.full()
# indexer.block(left_on='State', right_on='Provider State')

# Use sortedneighbor as a good option if data is not clean
indexer.sortedneighbourhood(left_on='name', right_on='name')
candidates = indexer.index(muaban_data, okxe_data)
compare = recordlinkage.Compare()
# compare.exact('City', 'Provider City', label='City')
compare.string('name','name', method='levenshtein',threshold=0.8,label='Name')
compare.string('address', 'address', method='jarowinkler', threshold=0.7, label='address')
compare.string('Tình trạng', 'used_status', method='jarowinkler', threshold=0.5, label='Tình trạng')
compare.string('Hãng xe', 'brand', method='jarowinkler', threshold=0.5, label='Hãng xe')
compare.string('Dòng xe', 'model', method='jarowinkler', threshold=0.5, label='Dòng xe')
features = compare.compute(candidates, muaban_data,
                           okxe_data)
# Get the potential matches
potential_matches = features[features.sum(axis=1) > 3].reset_index()
potential_matches

,level_0,level_1,Name,address,Tình trạng,Hãng xe,Dòng xe


In [141]:
all = all.append(okxe_data[~okxe_data.index.isin(potential_matches['level_1'])].rename(columns=okxe_inv).rename(columns=inverse_dict_field))
all

/tmp/ipykernel_55707/1937918524.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(okxe_data[~okxe_data.index.isin(potential_matches['level_1'])].rename(columns=okxe_inv).rename(columns=inverse_dict_field))


,name,price,address,url,brand,registerYear,status,type,origin,capacity,...,used_distance,Giảm xóc - Phuộc,Phanh,Động cơ,Đồng hồ,Khoá,Hệ thống đèn,Hệ thống tự động,Công nghệ kết nối,Khác
0,CẦN BÁN XE MÁY HONDA VISION 110 ĐỜI 2022,15000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Chuyên thu mua các loại xe máy cũ và mới, xe h...",0,"Quận Đống Đa, Hà Nội",https://muaban.net/xe-may/mua-xe-may/chuyen-th...,-,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HEAD SƠN TÙNG 1 - TRỢ GIÁ MUA XE TƯNG BỪNG-VUI...,31990000,"Huyện Hoài Đức, Hà Nội",https://muaban.net/xe-may/ban-xe-may/head-son-...,Honda,-,Mới,Tay côn/Moto,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cần thanh lý lô xe honda AirBlade 150 nhập khẩu,19000000,"TP. Thủ Đức - Quận 9, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-thanh...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CẦN BÁN XE MÁY HONDA SH150 ĐỜI 2022,32000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2023.0,Mới,Tay ga,Nước khác,101 - 175 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,Honda Winner 150,17500000,Tp. Hà Nội,https://okxe.vn/products/honda-winner-150-1697565,Honda,NaN,Đã sử dụng,2,NaN,NaN,...,0-5000km,Phuộc ống lồng,"Phanh ABS,Phanh thủy lực",Phun xăng điện tử Fi / PGM-FI,-,Khóa chống trộm,-,-,-,-
328,Bajaj Pulsar 125,31000000,Tp. Hà Nội,https://okxe.vn/products/bajaj-pulsar-125-1697564,Bajaj,NaN,Đã sử dụng,2,NaN,NaN,...,0-5000km,-,-,-,-,-,-,-,-,-
329,Honda Air Blade 125 FI,22000000,Tp. Hà Nội,https://okxe.vn/products/honda-air-blade-125-f...,Honda,NaN,Đã sử dụng,1,NaN,NaN,...,0-5000km,-,"Phanh cơ,Phanh Đĩa",Phun xăng điện tử Fi / PGM-FI,-,-,-,Hệ thống ngắt động cơ tạm thời - Idling Stop,-,-
330,Honda Click 110,6500000,Tp. Hà Nội,https://okxe.vn/products/honda-click-110-1697561,Honda,NaN,Đã sử dụng,2,NaN,NaN,...,0-5000km,-,"Phanh thắng đùm/ tang trống,Phanh Đĩa",Phun xăng điện tử Fi / PGM-FI,-,-,-,Hệ thống ngắt động cơ tạm thời - Idling Stop,-,-


In [142]:
xemayhoangkien_inv

{'price': 'price',
 'name': 'name',
 'url': 'url',
 'Năm đăng ký': 'Năm đăng ký',
 'Màu sắc': 'Màu xe',
 'Dung tích': 'Dung tích xe máy',
 'Thương hiệu': 'Hãng xe',
 'Loại': 'Loại xe máy'}

In [169]:
# Build the indexer
indexer = recordlinkage.Index()
# Can use full or block
# indexer.full()
# indexer.block(left_on='State', right_on='Provider State')

# Use sortedneighbor as a good option if data is not clean
indexer.sortedneighbourhood(left_on='name', right_on='name')
candidates = indexer.index(muaban_data, okxe_data)
compare = recordlinkage.Compare()
# compare.exact('City', 'Provider City', label='City')
compare.string('name','name', method='levenshtein',threshold=0.8,label='Name')
compare.exact('Năm đăng ký', 'Năm đăng ký', label='Năm đăng ký')
compare.string('Màu xe', 'Màu sắc', method='levenshtein', threshold=0.5, label='Màu xe')
compare.string('Dung tích xe máy', 'Dung tích', method='jarowinkler', threshold=0.7, label='Dung tích xe máy')
compare.string('Hãng xe', 'Thương hiệu', method='jarowinkler', threshold=0.5, label='Hãng xe')
compare.string('Loại xe máy', 'Loại', method='jarowinkler', threshold=0.5, label='Loại xe máy')
features = compare.compute(candidates, muaban_data,
                           xemayhoangkien_data)
# Get the potential matches
potential_matches = features[features.sum(axis=1) > 3].reset_index()
potential_matches

,level_0,level_1,Name,Năm đăng ký,Màu xe,Dung tích xe máy,Hãng xe,Loại xe máy


In [171]:
all = all.append(xemayhoangkien_data[~xemayhoangkien_data.index.isin(potential_matches['level_1'])].rename(columns=xemayhoangkien_inv).rename(columns=inverse_dict_field))
all

/tmp/ipykernel_55707/1265139995.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(xemayhoangkien_data[~xemayhoangkien_data.index.isin(potential_matches['level_1'])].rename(columns=xemayhoangkien_inv).rename(columns=inverse_dict_field))


,name,price,address,url,brand,registerYear,status,type,origin,capacity,...,Giảm xóc - Phuộc,Phanh,Động cơ,Đồng hồ,Khoá,Hệ thống đèn,Hệ thống tự động,Công nghệ kết nối,Khác,MSP
0,CẦN BÁN XE MÁY HONDA VISION 110 ĐỜI 2022,15000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Chuyên thu mua các loại xe máy cũ và mới, xe h...",0,"Quận Đống Đa, Hà Nội",https://muaban.net/xe-may/mua-xe-may/chuyen-th...,-,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HEAD SƠN TÙNG 1 - TRỢ GIÁ MUA XE TƯNG BỪNG-VUI...,31990000,"Huyện Hoài Đức, Hà Nội",https://muaban.net/xe-may/ban-xe-may/head-son-...,Honda,-,Mới,Tay côn/Moto,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cần thanh lý lô xe honda AirBlade 150 nhập khẩu,19000000,"TP. Thủ Đức - Quận 9, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-thanh...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CẦN BÁN XE MÁY HONDA SH150 ĐỜI 2022,32000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2023.0,Mới,Tay ga,Nước khác,101 - 175 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Vision 29V-058CG,16300000,52 Chùa Hà - Cầu Giấy - Hà Nội,https://xemayhoangkien.com/vision-29v-058cg,Honda,2012,NaN,Vision,NaN,110cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29V-058CG
779,Lead 29V-580BF,39900000,52 Chùa Hà - Cầu Giấy - Hà Nội,https://xemayhoangkien.com/lead-29v-580bf,Honda,2022,NaN,Lead,NaN,125cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29V-580BF
780,Winner 18B-937CF,18500000,52 Chùa Hà - Cầu Giấy - Hà Nội,https://xemayhoangkien.com/winner-18b-937cf,Honda,2017,NaN,Winner,NaN,150cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18B-937CF
781,Shmode 29L-553AG,57500000,52 Chùa Hà - Cầu Giấy - Hà Nội,https://xemayhoangkien.com/shmode-29l-553ag,Honda,2022,NaN,SH Mode,NaN,125cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shmode 29L-553AG


In [175]:
all[['name', 'price', 'address', 'url', 'brand', 'registerYear', 'status', 'type', 'origin', 'capacity', 'color', 'vehicleType', 'km', 'imageUrl']].to_csv('./final_data.csv')